<a href="https://colab.research.google.com/github/srJboca/segmentacion/blob/main/ES/1.%20Exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Exploración de Datos de Clientes y Facturación de Gas

## Introducción

Bienvenido a este tutorial sobre exploración de datos. Trabajaremos con un conjunto de datos que simula la información de una empresa de distribución de gas. El objetivo es limpiar, combinar y explorar estos datos para entender mejor a los clientes, sus consumos y comportamientos de pago. Este proceso es fundamental antes de realizar tareas más avanzadas como la segmentación de clientes, modelado predictivo de consumo o análisis de morosidad.

Los datos se dividen en cuatro archivos:
1.  `clientes.parquet`: Información sobre los clientes (contratos, datos demográficos).
2.  `facturas.parquet`: Detalles de las facturas emitidas (consumo, fechas).
3.  `precios_gas.parquet`: Precios del gas por m³ según estrato, año y mes.
4.  `recaudo.parquet`: Información sobre los pagos realizados de las facturas.

## Fase 1: Configuración del Entorno y Carga de Datos

### 1.1 Importación de Librerías

Primero, importaremos las librerías necesarias.
* `pandas` para la manipulación y análisis de datos.
* `matplotlib.pyplot` y `seaborn` para la visualización de datos.
* `warnings` para gestionar las advertencias que puedan surgir.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Configuraciones para visualizaciones
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### 1.2 Descarga de los Archivos de Datos

Descargaremos los archivos Parquet desde el repositorio de GitHub.

In [ ]:
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/clientes.parquet
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/facturas.parquet
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/precios_gas.parquet
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/recaudo.parquet

### 1.3 Carga de Datos en DataFrames de Pandas

Ahora, cargaremos cada archivo Parquet en un DataFrame de Pandas.

In [ ]:
df_clientes = pd.read_parquet('clientes.parquet')
df_facturas = pd.read_parquet('facturas.parquet')
df_recaudo = pd.read_parquet('recaudo.parquet')
df_precios_gas = pd.read_parquet('precios_gas.parquet')

## Fase 2: Inspección Inicial de los Datos

Realizaremos una inspección básica de cada DataFrame para entender su estructura, tipos de datos, y verificar la presencia de valores nulos o estadísticas descriptivas iniciales.

### 2.1 DataFrame de Clientes (`df_clientes`)

In [ ]:
print("--- Información de df_clientes ---")
df_clientes.info()
print("\n--- Primeras 5 filas de df_clientes ---")
print(df_clientes.head())
print("\n--- Valores nulos en df_clientes ---")
print(df_clientes.isnull().sum())
print("\n--- Estadísticas descriptivas de df_clientes ---")
print(df_clientes.describe(include='all'))
print("\n--- Conteo de valores únicos por columna en df_clientes ---")
for col in df_clientes.columns:
    print(f"Columna '{col}': {df_clientes[col].nunique()} valores únicos")

**Observaciones sobre `df_clientes`:**
* Contiene información personal y contractual de los clientes.
* Columnas como `Ciudad` y `Estrato socioeconomico` son categóricas y podrían ser importantes para la segmentación.
* Las fechas están como objetos, necesitaremos convertirlas a tipo `datetime`.
* `Numero de contrato` parece ser el identificador único.

### 2.2 DataFrame de Facturas (`df_facturas`)

In [ ]:
print("--- Información de df_facturas ---")
df_facturas.info()
print("\n--- Primeras 5 filas de df_facturas ---")
print(df_facturas.head())
print("\n--- Valores nulos en df_facturas ---")
print(df_facturas.isnull().sum())
print("\n--- Estadísticas descriptivas de df_facturas ---")
print(df_facturas.describe(include='all'))
print("\n--- Conteo de valores únicos por columna en df_facturas ---")
for col in df_facturas.columns:
    print(f"Columna '{col}': {df_facturas[col].nunique()} valores únicos")

**Observaciones sobre `df_facturas`:**
* Contiene detalles de cada factura, incluyendo consumo y fechas relevantes.
* `Numero de factura` es el identificador de la factura y `Numero de contrato` la vincula al cliente.
* `Consumo (m3)` es una variable numérica clave.
* Las fechas también están como objetos.

### 2.3 DataFrame de Recaudo (`df_recaudo`)

In [ ]:
print("--- Información de df_recaudo ---")
df_recaudo.info()
print("\n--- Primeras 5 filas de df_recaudo ---")
print(df_recaudo.head())
print("\n--- Valores nulos en df_recaudo ---")
print(df_recaudo.isnull().sum())
print("\n--- Estadísticas descriptivas de df_recaudo ---")
print(df_recaudo.describe(include='all'))
print("\n--- Conteo de valores únicos por columna en df_recaudo ---")
for col in df_recaudo.columns:
    print(f"Columna '{col}': {df_recaudo[col].nunique()} valores únicos")

**Observaciones sobre `df_recaudo`:**
* Registra la fecha en que se realizó el pago de una factura.
* Se une a `df_facturas` mediante `Numero de factura`.
* `Fecha de Pago Real` necesitará conversión a `datetime`.

### 2.4 DataFrame de Precios del Gas (`df_precios_gas`)

In [ ]:
print("--- Información de df_precios_gas ---")
df_precios_gas.info()
print("\n--- Primeras 5 filas de df_precios_gas ---")
print(df_precios_gas.head())
print("\n--- Valores nulos en df_precios_gas ---")
print(df_precios_gas.isnull().sum())
print("\n--- Estadísticas descriptivas de df_precios_gas ---")
print(df_precios_gas.describe(include='all'))
print("\n--- Conteo de valores únicos por columna en df_precios_gas ---")
for col in df_precios_gas.columns:
    print(f"Columna '{col}': {df_precios_gas[col].nunique()} valores únicos")

**Observaciones sobre `df_precios_gas`:**
* Define el precio del m³ de gas según `Año`, `Mes` y `Estrato`.
* `Precio m3 (COP)` es el valor numérico del precio.
* `Estrato` es categórico.

## Fase 3: Combinación (Merge) de los Datos

Para tener una visión consolidada, combinaremos estos DataFrames.

### 3.1 Unir Facturas con Información de Clientes

Unimos `df_facturas` con `df_clientes` usando `Numero de contrato` como clave.

In [ ]:
df_factura_cliente = pd.merge(df_facturas, df_clientes, on='Numero de contrato', how='left')
print("--- Información de df_factura_cliente (Facturas + Clientes) ---")
df_factura_cliente.info()
print("\n--- Primeras 5 filas de df_factura_cliente ---")
print(df_factura_cliente.head())
print(f"\nShape de df_facturas: {df_facturas.shape}")
print(f"Shape de df_factura_cliente: {df_factura_cliente.shape}")

**Verificación:** El número de filas debería ser igual al de `df_facturas` si cada factura tiene un cliente correspondiente (usando `how='left'`). Si aumenta, podría indicar duplicados en `df_clientes` por `Numero de contrato`, lo cual no debería ser.

### 3.2 Añadir Precios a las Facturas

Ahora unimos `df_factura_cliente` con `df_precios_gas`. La unión se hace usando `Año`, `Mes` y el estrato del cliente.

In [ ]:
df_factura_cliente_precio = pd.merge(df_factura_cliente,
                                     df_precios_gas,
                                     left_on=['Año', 'Mes', 'Estrato socioeconomico'],
                                     right_on=['Año', 'Mes', 'Estrato'],
                                     how='left')

print("--- Información de df_factura_cliente_precio (Facturas + Clientes + Precios) ---")
df_factura_cliente_precio.info()
print("\n--- Primeras 5 filas de df_factura_cliente_precio ---")
print(df_factura_cliente_precio.head())
print(f"\nShape de df_factura_cliente: {df_factura_cliente.shape}")
print(f"Shape de df_factura_cliente_precio: {df_factura_cliente_precio.shape}")

# Verificar si hay filas donde el precio no se pudo asignar (NaN en 'Precio m3 (COP)')
print("\n--- Filas sin precio asignado ---")
print(df_factura_cliente_precio[df_factura_cliente_precio['Precio m3 (COP)'].isnull()].head())

**Verificación y Posibles Problemas:**
* Nuevamente, el número de filas debería ser consistente. Si aumenta drásticamente, podría indicar que las claves de unión `['Año', 'Mes', 'Estrato socioeconomico']` no son únicas en `df_precios_gas` para una combinación dada, o que hay múltiples facturas para el mismo cliente en el mismo mes con el mismo estrato que se están mapeando (lo cual es esperado).
* La columna `Estrato` de `df_precios_gas` es redundante después del merge y puede ser eliminada.
* Es crucial verificar si hay facturas a las que no se les pudo asignar un precio. Esto podría ocurrir si alguna combinación de `Año, Mes, Estrato socioeconomico` en las facturas no existe en la tabla de precios.

### 3.3 Incorporar Información de Pagos (Recaudo)

Finalmente, unimos la información de pagos de `df_recaudo`.

In [ ]:
df_completo = pd.merge(df_factura_cliente_precio,
                       df_recaudo,
                       on='Numero de factura',
                       how='left') # Usamos left join para mantener todas las facturas, incluso si no tienen pago registrado

print("--- Información de df_completo (Todos los datos unidos) ---")
df_completo.info()
print("\n--- Primeras 5 filas de df_completo ---")
print(df_completo.head())
print(f"\nShape de df_factura_cliente_precio: {df_factura_cliente_precio.shape}")
print(f"Shape de df_completo: {df_completo.shape}")

# Verificar facturas sin fecha de pago real
print("\n--- Facturas sin fecha de pago real (NaN en 'Fecha de Pago Real') ---")
print(df_completo[df_completo['Fecha de Pago Real'].isnull()][['Numero de factura', 'Fecha de Emision', 'Fecha de Pago Oportuno']].head())

**Verificación:**
* El número de filas debería mantenerse o aumentar si una factura tiene múltiples registros de pago (poco común, a menos que sean pagos parciales no consolidados). Un `left join` asegura que todas las facturas se mantengan.
* Las facturas sin un `Fecha de Pago Real` correspondiente tendrán `NaN` en esa columna. Esto es esperado para facturas no pagadas o cuyo pago aún no se ha registrado.

### 3.4 Selección de Columnas y Manejo de Duplicados

El DataFrame original `df_analisis` seleccionaba un subconjunto de columnas. Vamos a replicar esto y luego manejar posibles duplicados que se hayan generado en los merges, especialmente si las claves no eran perfectamente únicas o si la misma información se unió múltiples veces.

In [ ]:
columnas_seleccionadas = [
    'Numero de factura', 'Numero de contrato', 'Fecha de Emision', 'Consumo (m3)',
    'Fecha de Pago Oportuno', 'Fecha de Lectura', 'Fecha de Suspension Estimada',
    'Ciudad', 'Estrato socioeconomico',
    'Precio m3 (COP)', 'Fecha de Pago Real'
]

if 'Estrato' in df_completo.columns and 'Estrato socioeconomico' in df_completo.columns:
    df_completo = df_completo.drop(columns=['Estrato'])

df_analisis = df_completo[columnas_seleccionadas].copy()

print("--- Primeras filas de df_analisis (antes de tratar duplicados) ---")
print(df_analisis.head())
print(f"\nShape de df_analisis antes de drop_duplicates: {df_analisis.shape}")

num_duplicados_antes = df_analisis.duplicated().sum()
print(f"Número de filas duplicadas exactas antes: {num_duplicados_antes}")

df_analisis = df_analisis.drop_duplicates()
print(f"\nShape de df_analisis después de drop_duplicates: {df_analisis.shape}")
num_duplicados_despues = df_analisis.duplicated().sum()
print(f"Número de filas duplicadas exactas después: {num_duplicados_despues}")

print("\n--- Primeras filas de df_analisis (después de tratar duplicados) ---")
print(df_analisis.head())
df_analisis.info()

**Nota sobre Duplicados:**
El notebook original mostraba filas duplicadas después de los merges. El paso `drop_duplicates()` es crucial para asegurar la integridad del análisis. Es importante investigar *por qué* se generaron duplicados (e.g., ¿claves de merge no únicas? ¿datos fuente con duplicados?). Por ahora, los hemos eliminado.

## Fase 4: Ingeniería de Características (Feature Engineering)

Crearemos nuevas columnas a partir de las existentes para enriquecer nuestro análisis.

### 4.1 Conversión de Fechas

Convertimos las columnas de fecha a formato `datetime`.

In [ ]:
date_cols = ['Fecha de Emision', 'Fecha de Pago Oportuno', 'Fecha de Lectura', 'Fecha de Pago Real', 'Fecha de Suspension Estimada']
for col in date_cols:
    df_analisis[col] = pd.to_datetime(df_analisis[col], errors='coerce')

print("--- Tipos de datos después de convertir fechas ---")
df_analisis.info()

### 4.2 Cálculo del Monto de la Factura (Precio por Consumo)

In [ ]:
df_analisis['Precio por Consumo'] = df_analisis['Precio m3 (COP)'] * df_analisis['Consumo (m3)']
print("\n--- df_analisis con 'Precio por Consumo' ---")
print(df_analisis[['Consumo (m3)', 'Precio m3 (COP)', 'Precio por Consumo']].head())

### 4.3 Cálculo de Diferencias Temporales

Calcularemos los días entre diferentes eventos clave.

In [ ]:
df_analisis['Dias_Emision_PagoOportuno'] = (df_analisis['Fecha de Pago Oportuno'] - df_analisis['Fecha de Emision']).dt.days
df_analisis['Dias_Lectura_Emision'] = (df_analisis['Fecha de Emision'] - df_analisis['Fecha de Lectura']).dt.days
df_analisis['Dias_PagoOportuno_PagoReal'] = (df_analisis['Fecha de Pago Real'] - df_analisis['Fecha de Pago Oportuno']).dt.days

print("\n--- df_analisis con nuevas características de días ---")
print(df_analisis[['Fecha de Emision', 'Fecha de Pago Oportuno', 'Fecha de Lectura', 'Fecha de Pago Real',
                   'Dias_Emision_PagoOportuno', 'Dias_Lectura_Emision', 'Dias_PagoOportuno_PagoReal']].head())

### 4.4 Identificación de Mora en Pagos

Creamos una columna binaria `Mora`: 1 si el pago se realizó después de la fecha de pago oportuno, 0 si se pagó a tiempo o antes. Se considera 0 si no hay fecha de pago real (NaT).

In [ ]:
df_analisis['Mora'] = 0
df_analisis.loc[df_analisis['Dias_PagoOportuno_PagoReal'] > 0, 'Mora'] = 1
df_analisis.loc[df_analisis['Dias_PagoOportuno_PagoReal'].isnull(), 'Mora'] = 0

print("\n--- df_analisis con la columna 'Mora' ---")
print(df_analisis[['Fecha de Pago Oportuno', 'Fecha de Pago Real', 'Dias_PagoOportuno_PagoReal', 'Mora']].head(10))
print("\nConteo de valores en 'Mora':")
print(df_analisis['Mora'].value_counts(dropna=False))

## Fase 5: Exploración Detallada del DataFrame Consolidado (`df_analisis`)

Ahora que tenemos un DataFrame limpio y enriquecido, podemos explorarlo más a fondo.

### 5.1 Resumen General y Valores Nulos

In [ ]:
print("--- Información general de df_analisis ---")
df_analisis.info()

print("\n--- Valores nulos en df_analisis ---")
print(df_analisis.isnull().sum())

print("\n--- Estadísticas descriptivas de df_analisis ---")
print(df_analisis.describe(include='all'))

**Observaciones sobre valores nulos:**
* `Fecha de Suspension Estimada`: Puede tener nulos si no todas las facturas tienen esta fecha.
* `Precio m3 (COP)` y `Precio por Consumo`: Nulos aquí indicarían problemas en el merge con `df_precios_gas` o datos faltantes en la tabla de precios.
* `Fecha de Pago Real` y `Dias_PagoOportuno_PagoReal`: Nulos son esperados para facturas no pagadas.

Es importante decidir cómo tratar estos nulos. Para algunas variables, podrían imputarse, para otras (como `Fecha de Pago Real`), su ausencia es informativa.

### 5.2 Distribución de Variables Numéricas Clave

In [ ]:
numerical_cols_to_plot = ['Consumo (m3)', 'Precio m3 (COP)', 'Precio por Consumo',
                           'Dias_Emision_PagoOportuno', 'Dias_Lectura_Emision', 'Dias_PagoOportuno_PagoReal']

for col in numerical_cols_to_plot:
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    sns.histplot(df_analisis[col].dropna(), kde=True, bins=30) # dropna() para evitar errores con NaT/NaN en histplot
    plt.title(f'Distribución de {col}')

    plt.subplot(1, 2, 2)
    sns.boxplot(y=df_analisis[col].dropna())
    plt.title(f'Boxplot de {col}')

    plt.tight_layout()
    plt.show()
    print(f"Estadísticas para {col}:\n{df_analisis[col].describe()}\n")

**Interpretación de distribuciones:**
* **Consumo (m3):** Observar la forma de la distribución. ¿Es simétrica, sesgada? ¿Hay outliers (consumos muy altos o bajos)?
* **Precio m3 (COP):** ¿Varía mucho? Esto dependerá de los estratos y la evolución temporal.
* **Precio por Consumo:** Similar al consumo, pero escalado por el precio.
* **Variables de Días:**
    * `Dias_Emision_PagoOportuno`: Plazo para pagar. ¿Es constante?
    * `Dias_Lectura_Emision`: Tiempo entre lectura y emisión. ¿Es consistente?
    * `Dias_PagoOportuno_PagoReal`: Valores positivos indican mora. Negativos, pago adelantado. Cero, pago el día oportuno. La alta concentración en valores negativos o cero en el head del notebook original sugiere pagos puntuales o adelantados para esos casos. El boxplot puede mostrar la magnitud de la mora o adelanto.

### 5.3 Distribución de Variables Categóricas Clave

In [ ]:
categorical_cols_to_plot = ['Ciudad', 'Estrato socioeconomico', 'Mora']

for col in categorical_cols_to_plot:
    plt.figure(figsize=(8, 5))
    sns.countplot(data=df_analisis, x=col, order=df_analisis[col].value_counts(dropna=False).index)
    plt.title(f'Distribución de {col}')
    plt.xticks(rotation=45)
    plt.show()
    print(f"Conteo para {col}:\n{df_analisis[col].value_counts(dropna=False)}\n")

**Interpretación de distribuciones categóricas:**
* **Ciudad:** ¿Cómo se distribuyen las facturas/clientes por ciudad?
* **Estrato socioeconomico:** ¿Qué estratos son los más comunes?
* **Mora:** ¿Qué proporción de facturas se pagan con mora?

### 5.4 Análisis de Correlación (Variables Numéricas)

In [ ]:
numerical_df = df_analisis.select_dtypes(include=['number'])

plt.figure(figsize=(12, 8))
correlation_matrix = numerical_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlación de Variables Numéricas')
plt.show()

print("\n--- Pares con mayor correlación (absoluta) ---")
corr_pairs = correlation_matrix.unstack()
sorted_pairs = corr_pairs.sort_values(kind="quicksort", ascending=False)
unique_corr_pairs = sorted_pairs[(abs(sorted_pairs) < 1) & (abs(sorted_pairs) > 0.5)].drop_duplicates() # Filtrar > 0.5 para relevancia
print(unique_corr_pairs.head(10))

**Interpretación de la Matriz de Correlación:**
* Busca coeficientes de correlación cercanos a 1 (correlación positiva fuerte) o -1 (correlación negativa fuerte).
* Por ejemplo, es esperable una alta correlación entre `Consumo (m3)` y `Precio por Consumo`.
* Correlaciones inesperadas pueden revelar insights interesantes.

### 5.5 Relaciones entre Variables

Exploremos algunas relaciones específicas.

#### 5.5.1 Consumo promedio por Estrato y Ciudad

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(data=df_analisis, x='Estrato socioeconomico', y='Consumo (m3)', hue='Ciudad', estimator=pd.Series.mean, errorbar=None, order=sorted(df_analisis['Estrato socioeconomico'].dropna().unique()))
plt.title('Consumo Promedio (m3) por Estrato Socioeconómico y Ciudad')
plt.ylabel('Consumo Promedio (m3)')
plt.xlabel('Estrato Socioeconómico')
plt.xticks(rotation=45)
plt.legend(title='Ciudad')
plt.show()

plt.figure(figsize=(14, 7))
sns.boxplot(data=df_analisis, x='Estrato socioeconomico', y='Consumo (m3)', hue='Ciudad', order=sorted(df_analisis['Estrato socioeconomico'].dropna().unique()))
plt.title('Distribución del Consumo (m3) por Estrato Socioeconómico y Ciudad')
plt.ylabel('Consumo (m3)')
plt.xlabel('Estrato Socioeconómico')
plt.xticks(rotation=45)
plt.legend(title='Ciudad')
plt.show()

#### 5.5.2 Tasa de Mora por Estrato y Ciudad

In [ ]:
mora_por_estrato_ciudad = df_analisis.groupby(['Estrato socioeconomico', 'Ciudad'])['Mora'].mean().reset_index()
mora_por_estrato_ciudad = mora_por_estrato_ciudad.rename(columns={'Mora': 'Tasa de Mora'})

plt.figure(figsize=(12, 6))
sns.barplot(data=mora_por_estrato_ciudad, x='Estrato socioeconomico', y='Tasa de Mora', hue='Ciudad', order=sorted(mora_por_estrato_ciudad['Estrato socioeconomico'].dropna().unique()))
plt.title('Tasa de Mora por Estrato Socioeconómico y Ciudad')
plt.ylabel('Tasa de Mora (Proporción)')
plt.xlabel('Estrato Socioeconómico')
plt.xticks(rotation=45)
plt.legend(title='Ciudad')
plt.show()

### 5.6 Análisis Temporal (Ejemplo: Consumo a lo largo del tiempo)

In [ ]:
df_temporal = df_analisis.set_index('Fecha de Emision').sort_index()

consumo_mensual = df_temporal['Consumo (m3)'].resample('ME').mean()

plt.figure(figsize=(14, 7))
consumo_mensual.plot(marker='o', linestyle='-')
plt.title('Consumo Promedio Mensual de Gas (m3) a lo largo del Tiempo')
plt.xlabel('Fecha de Emisión (Mes)')
plt.ylabel('Consumo Promedio (m3)')
plt.grid(True)
plt.show()

facturas_mensuales = df_temporal.resample('ME').size()
plt.figure(figsize=(14, 7))
facturas_mensuales.plot(kind='bar')
plt.title('Número de Facturas Emitidas por Mes')
plt.xlabel('Fecha de Emisión (Mes)')
plt.ylabel('Número de Facturas')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

**Interpretación del Análisis Temporal:**
* ¿Hay tendencias en el consumo (aumento, disminución)?
* ¿Se observa estacionalidad (picos en ciertos meses)?
* La cantidad de facturas puede indicar crecimiento de la base de clientes o fluctuaciones en la facturación.

## Fase 6: Conclusiones de la Exploración y Próximos Pasos

En esta fase exploratoria, hemos:
1.  Cargado y realizado una inspección inicial de cuatro fuentes de datos.
2.  Combinado los datos en un DataFrame unificado (`df_analisis`).
3.  Identificado y manejado filas duplicadas que surgieron durante los merges.
4.  Realizado ingeniería de características, creando nuevas variables como `Precio por Consumo`, diferencias temporales y un indicador de `Mora`.
5.  Explorado las distribuciones de variables numéricas y categóricas clave.
6.  Analizado correlaciones y relaciones entre variables (e.g., consumo por estrato, mora por estrato).
7.  Realizado un breve análisis temporal del consumo.